## HuggingFace BERT Model Deployment

In [1]:
import sagemaker
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import RealTimePredictor, json_serializer, json_deserializer

role = sagemaker.get_execution_role()
# model_artefact = '<your-model-s3-path>'
model_artefact = 's3://sagemaker-eu-west-1-366243680492/pytorch-training-2020-06-14-21-03-05-418/output/model.tar.gz'

In [2]:
class SentimentAnalysis(RealTimePredictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super().__init__(endpoint_name, sagemaker_session=sagemaker_session, serializer=json_serializer, 
                         deserializer=json_deserializer, content_type='application/json')

model=PyTorchModel(model_data=model_artefact,
                   name=name_from_base('bert-model'),
                   role=role, 
                   entry_point='predictor.py',
                   source_dir='source_dir',
                   framework_version='1.5.0',
                   predictor_cls=SentimentAnalysis)

In [3]:
predictor = model.deploy(initial_instance_count=1, instance_type='ml.p3.2xlarge')

-----------------!

In [4]:
test_data = {"text": 'I love completing my todos! Best app ever!!!'}
print(test_data)

{'text': 'I love completing my todos! Best app ever!!!'}


In [5]:
prediction = predictor.predict(test_data)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (500) from model with message "Object of type 'Tensor' is not JSON serializable". See https://eu-west-1.console.aws.amazon.com/cloudwatch/home?region=eu-west-1#logEventViewer:group=/aws/sagemaker/Endpoints/bert-model-2020-06-14-22-10-51-938 in account 366243680492 for more information.

In [ ]:
class_names = ['negative', 'neutral', 'positive']
print(f'Review text: {review_text}')
print(f'Sentiment  : {class_names[prediction]}')